# Homework 1: Preprocessing and Text Classification

Student Name:Shengyi Zhao

Student ID:990160

# General Info

<b>Due date</b>: Sunday, 5 Apr 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to classify whether a tweet constitutes a rumour event. This homework involves writing code to preprocess data and perform text classification.

# 1. Preprocessing (5 marks)

**Instructions**: Run the code below to download the tweet corpus for the assignment. Note: the download may take some time. **No implementation is needed.**

In [1]:
import requests
import os
from pathlib import Path

fname = 'rumour-data.tgz'
data_dir = os.path.splitext(fname)[0] #'rumour-data'

my_file = Path(fname)
if not my_file.is_file():
    url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
    r = requests.get(url)

    #Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)
        
print("Done. File downloaded:", my_file)


Done. File downloaded: rumour-data.tgz


**Instructions**: Run the code to extract the zip file. Note: the extraction may take a minute or two. **No implementation is needed.**

In [2]:
import tarfile

#decompress rumour-data.tgz
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

#remove superfluous files (e.g. .DS_store)
extra_files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if (file.startswith(".")):
            extra_files.append(os.path.join(r, file))
for f in extra_files:
    os.remove(f)

print("Extraction done.")

Extraction done.


### Question 1 (1.0 mark)

**Instructions**: The corpus data is in the *rumour-data* folder. It contains 2 sub-folders: *non-rumours* and *rumours*. As the names suggest, *rumours* contains all rumour-propagating tweets, while *non-rumours* has normal tweets. Within  *rumours* and *non-rumours*, you'll find some sub-folders, each named with an ID. Each of these IDs constitutes an 'event', where an event is defined as consisting a **source tweet** and its **reactions**.

An illustration of the folder structure is given below:

    rumour-data
        - rumours
            - 498254340310966273
                - reactions
                    - 498254340310966273.json
                    - 498260814487642112.json
                - source-tweet
                    - 498254340310966273.json
        - non-rumours

Now we need to gather the tweet messages for rumours and non-rumour events. As the individual tweets are stored in json format, we need to use a json parser to parse and collect the actual tweet message. The function `get_tweet_text_from_json(file_path)` is provided to do that.

**Task**: Complete the `get_events(event_dir)` function. The function should return **a list of events** for a particular class of tweets (e.g. rumours), and each event should contain the source tweet message and all reaction tweet messages.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [3]:
import json

def get_tweet_text_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data["text"]
    
def get_events(event_dir):
    event_list = []
    for event in sorted(os.listdir(event_dir)):
        ###
        # Your answer BEGINS HERE
        ###
        content=''
        for subevent in sorted(os.listdir(os.path.join(event_dir,event,"reactions"))):
            content+=get_tweet_text_from_json(os.path.join(event_dir,event,"reactions",subevent))
        for subevent in sorted(os.listdir(os.path.join(event_dir,event,"source-tweet"))):
            content+=get_tweet_text_from_json(os.path.join(event_dir,event,"source-tweet",subevent))
        event_list.append(content)
        
        ###
        # Your answer ENDS HERE
        ###
        
    return event_list
    
#a list of events, and each event is a list of tweets (source tweet + reactions)    
rumour_events = get_events(os.path.join(data_dir, "rumours"))
nonrumour_events = get_events(os.path.join(data_dir, "non-rumours"))

print("Number of rumour events =", len(rumour_events))
print("Number of non-rumour events =", len(nonrumour_events))

Number of rumour events = 500
Number of non-rumour events = 1000


**For your testing:**

In [4]:
assert(len(rumour_events) == 500)
assert(len(nonrumour_events) == 1000)

### Question 2 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); and (2) remove stopwords (based on NLTK `stopwords`).

**Task**: Complete the `preprocess_events(event)` function. The function takes **a list of events** as input, and returns **a list of preprocessed events**. Each preprocessed event should have a dictionary of words and frequencies.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [5]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

def preprocess_events(events):
    ###
    # Your answer BEGINS HERE
    ###
    events_list=[]
    for subevent in events:
        data=tt.tokenize(subevent)
        filtered_words = [word for word in data if word not in stopwords]
        dictionary = defaultdict(int)
        for word in filtered_words:
            dictionary[word] += 1
        events_list.append(dictionary)
    return events_list

    ###
    # Your answer ENDS HERE
    ###

preprocessed_rumour_events = preprocess_events(rumour_events)
preprocessed_nonrumour_events = preprocess_events(nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


**For your testing**:

In [6]:
assert(len(preprocessed_rumour_events) == 500)
assert(len(preprocessed_nonrumour_events) == 1000)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [7]:
def get_all_hashtags(events):
    hashtags = set([])
    for event in events:
        for word, frequency in event.items():
            if word.startswith("#"):
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(preprocessed_rumour_events + preprocessed_nonrumour_events)
print("Number of hashtags =", len(hashtags))

Number of hashtags = 1938


### Question 3 (2.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing a reversed version of the MaxMatch algorithm discussed in class, where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching. When lemmatising a word, you also need to provide the part-of-speech tag of the word. You should use `nltk.tag.pos_tag` for doing part-of-speech tagging.

Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenized hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list.

For example, given "#speakup", the algorithm should produce: \["#", "speak", "up"\]. And note that you do not need to delete the hashtag symbol ("#") from the tokenised outputs.

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing a reversed MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of word tokens".

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [8]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK

def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    for word in words:
        word=lemmatizer.lemmatize(word,get_wordnet_pos(word))
        word=word.lower()
    dict={}
    for tags in hashtags:
        list=[]
        length = 0
        tag=tags
        full=len(tags)
        while length < len(tags):         
              word = tags[length:len(tags)]
              lower_version = word.lower()
              st_version=lemmatizer.lemmatize(word)
              if length==(len(tags)- 1):
                 list.append(word)
                 tags=tags[0:length]
                 length=0
              elif (lemmatizer.lemmatize(lower_version,get_wordnet_pos(lower_version))) in words:  
                      list.append(st_version)  
                      tags = tags[0:length]  
                      length = 0 
              else:  
                    length = length+1  
        dict[tag]=list
    return dict
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

        
    ###
    # Your answer ENDS HERE
    ###


tokenized_hashtags = tokenize_hashtags(hashtags)

print(list(tokenized_hashtags.items())[:20])

[('#chills', ['chill', '#']), ('#kosher', ['kosher', '#']), ("#Erdoğan's", ['s', "'", 'an', 'ğ', 'do', 'Er', '#']), ('#thankyouNo-Fly', ['Fly', '-', 'No', 'you', 'thank', '#']), ('#pegida', ['da', 'i', 'peg', '#']), ('#dangerousnegroesThat', ['That', 'negro', 'dangerous', '#']), ('#Lies', ['Lies', '#']), ('#crazy', ['crazy', '#']), ('#RT', ['T', 'R', '#']), ('#Islamic', ['amic', 'l', 'Is', '#']), ('#a320', ['0', '2', '3', 'a', '#']), ('#MikeBrownDont', ['Dont', 'Brown', 'Mike', '#']), ('#FergusonMO', ['MO', 'son', 'u', 'erg', 'F', '#']), ('#MidlandsHonda', ['Honda', 'Midlands', '#']), ('#Music101', ['1', '0', '1', 'Music', '#']), ('#ftp', ['p', 't', 'f', '#']), ('#ModiSilenceMedia', ['Media', 'Silence', 'di', 'Mo', '#']), ('#Australia', ['a', 'li', 'stra', 'u', 'A', '#']), ('#heavyheart', ['heart', 'heavy', '#']), ('#WE', ['WE', '#'])]


**For your testing:**

In [9]:
assert(len(tokenized_hashtags) == len(hashtags))

### Question 4 (1.0 mark)

**Instructions**: Now that we have the tokenized hashtags, we need to go back and update the bag-of-words representation for each event.

**Task**: Complete the ``update_event_bow(events)`` function. The function takes **a list of preprocessed events**, and for each event, it looks for every hashtag it has and updates the bag-of-words dictionary with the tokenized hashtag tokens. Note: you do not need to delete the counts of the original hashtags when updating the bag-of-words (e.g., if a document has "#speakup":2 in its bag-of-words representation, you do not need to delete this hashtag and its counts).

In [10]:
def update_event_bow(events):
    ###
    # Your answer BEGINS HERE
    ###
    for event in events:
        hashtags = defaultdict(int)
        for word, frequency in event.items():
            if word.startswith("#"):
               tokenized_tag=tokenized_hashtags[word]
               for token_word in tokenized_tag:
                   hashtags[token_word]+=1
        event.update(hashtags)          

    ###
    # Your answer ENDS HERE
    ###
         
update_event_bow(preprocessed_rumour_events)
update_event_bow(preprocessed_nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict, given a tweet and its reactions, whether it is a rumour or not. The task here is to create training, development and test partitions from the preprocessed events and convert the bag-of-words representation into feature vectors.

**Task**: Using scikit-learn, create training, development and test partitions with a 60%/20%/20% ratio. Remember to preserve the ratio of rumour/non-rumour events for all your partitions. Next, turn the bag-of-words dictionary of each event into a feature vector, using scikit-learn `DictVectorizer`.

In [39]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()

###
# Your answer BEGINS HERE
###
from sklearn.model_selection import train_test_split  
import pandas as pd
label_rumour=[]
label_nonrumour=[]
for a in preprocessed_rumour_events:
    label_rumour.append(1)
for b in preprocessed_nonrumour_events:
    label_nonrumour.append(0)
train_rumour, other_rumour,train_label_rumour,other_label_rumour = train_test_split(preprocessed_rumour_events,label_rumour, test_size=0.6, random_state=0)
develop_rumour,test_rumour,develop_label_rumour,test_label_rumour =train_test_split(other_rumour,other_label_rumour,test_size=0.5,random_state=0)
train_nonrumour, other_nonrumour,train_label_nonrumour,other_label_nonrumour = train_test_split(preprocessed_nonrumour_events,label_nonrumour, test_size=0.6, random_state=0)
develop_nonrumour,test_nonrumour,develop_label_nonrumour,test_label_nonrumour=train_test_split(other_nonrumour,other_label_nonrumour,test_size=0.5,random_state=0)
all_train=train_rumour+train_nonrumour
all_train_label=train_label_rumour+train_label_nonrumour
all_develop =develop_rumour+develop_nonrumour
all_develop_label=develop_label_rumour+develop_label_nonrumour
all_test= test_rumour+test_nonrumour
all_test_label=test_label_rumour+test_label_nonrumour
all_data=all_train+all_develop+all_test
all_train_vector=vectorizer.fit_transform(all_train).toarray()
all_develop_vector=vectorizer.transform(all_develop)
all_test_vector=vectorizer.transform(all_test)

###
# Your answer ENDS HERE
###

print("Vocabulary size =", len(vectorizer.vocabulary_))

Vocabulary size = 20916


### Question 6 (2.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance for different hyper-parameter settings.

In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

###
# Your answer BEGINS HERE
###
from sklearn.pipeline import Pipeline
#nb

nb_model = MultinomialNB()
clf = Pipeline([('nb', nb_model)])

####   grid search start
best_score = 0
train_y = np.array(all_train_label)
develop_y=np.array(all_develop_label)
for nb_alpha in [0.001,0.005,0.008,0.009,0.01,0.011,0.012,0.02,0.1,1,2,10,100]:
        model = MultinomialNB(alpha=nb_alpha)
        model.fit(all_train_vector,train_y)
        score = model.score(all_develop_vector,develop_y)
        if score > best_score:
            best_score = score
            best_parameters = {'alpha':nb_alpha}
####   grid search end
print("Best score:{:.2f}".format(best_score))
print("Best parameters:{}".format(best_parameters))

#for LogisticRegression
lr_model = LogisticRegression(solver='lbfgs',multi_class='multinomial')
clf = Pipeline([('lr', lr_model)])

best_score2 = 0.0
for lr_c in [0.001,0.01,0.1,0.15,0.16,0.164,0.165,0.166,0.17,0.175,0.18,0.2,0.225,0.25,0.3,0.4,0.5,0.6,0.75,1,1.25,1.5,10,100]:
        model2 = LogisticRegression(solver='lbfgs',multi_class='multinomial',C=lr_c)
        model2.fit(all_train_vector,train_y)
        score2 = model2.score(all_develop_vector,develop_y)
        if score2 > best_score2:
            best_score2 = score2
            best_parameters2 = {'C':lr_c}

print("Best score:{:.2f}".format(best_score2))
print("Best parameters:{}".format(best_parameters2))

###
# Your answer ENDS HERE
###

Best score:0.81
Best parameters:{'alpha': 0.1}


C:\Users\New\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\New\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regress

Best score:0.79
Best parameters:{'C': 0.165}


C:\Users\New\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Using best sceore and parameters to store
after testing [0.001,0.005,0.008,0.009,0.01,0.011,0.012,0.02,0.1,1,2,10,100]
Best score:0.81
Best parameters of NB:{'alpha': 0.1}
after testing [0.001,0.01,0.1,0.15,0.16,0.164,0.165,0.166,0.17,0.175,0.18,0.2,0.225,0.25,0.3,0.4,0.5,0.6,0.75,1,1.25,1.5,10,100]
Best score:0.79
Best parameters of LogisticRegression:{'C': 0.165}

### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macro-averaged F-score for each classifier. Be sure to label your output.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using optimal hyper-parameter settings.

In [53]:
###
# Your answer BEGINS HERE
###
from sklearn import metrics

test_y = np.array(all_test_label)

model = MultinomialNB(**best_parameters)
model.fit(all_train_vector,train_y)
predict_y_nb = model.predict(all_test_vector)

model2 = LogisticRegression(**best_parameters2)
model2.fit(all_train_vector,train_y)
predict_y_lr = model2.predict(all_test_vector)

score1=metrics.precision_score(test_y, predict_y_nb, average='micro')
score2=metrics.precision_score(test_y, predict_y_lr, average='micro')
fscore1=metrics.f1_score(test_y, predict_y_nb, average='weighted')
fscore2=metrics.f1_score(test_y, predict_y_lr, average='weighted')

print("accuracy:{:.2f} of nb".format(score1))
print("accuracy:{:.2f} of lr".format(score2))
print("f-score: {:.2f} of nb".format(fscore1))
print("f-score: {:.2f} of lr".format(fscore2))

###
# Your answer ENDS HERE
###

accuracy:0.80 of nb
accuracy:0.79 of lr
f-score: 0.80 of nb
f-score: 0.78 of lr
